<a href="https://colab.research.google.com/github/kk25gb/AIdea-project-file/blob/Siamese-network/Indexer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 建立索引清單

In [204]:
import json
import csv
from urllib.request import urlopen
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import tqdm

In [224]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage2/vectorize_data.json'
response = urlopen(url)
data = json.loads(response.read())
df = pd.DataFrame(data).T

In [206]:
# url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage2/similarity_list_stage_2.json'
# response = urlopen(url)
# similarity_list = json.loads(response.read())

In [207]:
# similarity_list

In [208]:
# url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage2/column_filter.json'
# response = urlopen(url)
# column_filter = json.loads(response.read())

In [225]:
def Union_calculation(input):
  union = set()
  for values in list(input.values()):
    if len(union)==0:
      union = set(values)
    else:
      union = union & set(values)
  return union

In [226]:
similarity_list = {}
column_filter={}
for key,labels in data.items():
  row_1 = pd.DataFrame(labels).T
  mask = row_1 > 0
  filter = row_1[mask].dropna(axis=1).columns.tolist()
  column_filter[key]=filter
  filter_list = {}
  for i in filter:
    filter_list[i] = df.iloc[np.flatnonzero(df[i])].index.tolist()
  similarity_list[key] = list(Union_calculation(filter_list))

In [ ]:
# column_filter

In [ ]:
# similarity_list

In [213]:
index=3
object_list = df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]]
object_list.iloc[:].tolist()

[6, 14, 2, 3]

In [227]:
index = 3
# index = list(column_filter.keys()).index('10')
print("Object:",list(column_filter.keys())[index])
print("index:",column_filter[list(column_filter.keys())[index]])
print("values:",df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]].tolist())
print("same_index_rows:",similarity_list[list(similarity_list.keys())[index]])
index_filter = df[column_filter[list(column_filter.keys())[index]]].loc[similarity_list[list(similarity_list.keys())[index]]]
index_filter

Object: 1008
index: [29, 87, 370, 571]
values: [2, 2, 6, 11]
same_index_rows: ['573', '1008', '574', '128']


,29,87,370,571
573,2,2,5,9
1008,2,2,6,11
574,2,2,5,7
128,2,1,1,3


# 進行迭代分析

In [215]:
df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]].tolist()

[4, 3, 4, 3, 2]

In [ ]:
results = {}
for i in list(column_filter.keys()):
  index_object = column_filter[i]
  # print("Object:",i,index_object)
  compare_object = df[column_filter[i]].loc[i].tolist()
  results[i]={"key":index_object}
  compare_table = df[column_filter[i]].loc[similarity_list[i]]
  temp={}
  for key, value in zip(compare_table.iloc[:,:].index,compare_table.iloc[:,:].values):
    if key!=i:temp[key] = value.tolist()
  if len(temp)>0:
    results[i]["values"]=temp
  else:
    del results[i]
  # print('----------------------')
  # break
  

In [217]:
# compare_table.iloc[:,:].index

Index(['998'], dtype='object')

In [ ]:
# results

# 匯出檔案

##儲存為JSON格式

In [219]:
import json
with open('./results.json', 'w', encoding='utf8') as f:
    json.dump(results, f, ensure_ascii=False)

## 從Colab下載檔案到電腦

In [ ]:
from google.colab import files
files.download("/content/results.json")